In [1]:
import numpy as np
import pescador
import logging
import os

import torch
import torch.nn as nn
import torch.autograd as autograd
import torch.optim as optim

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from datetime import datetime

In [2]:
LOGGER = logging.getLogger('gbsd')
LOGGER.setLevel(logging.DEBUG)

In [3]:
torch.set_printoptions(sci_mode=False)
np.set_printoptions(suppress=True)

In [4]:
matplotlib.use('Agg')

In [5]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [6]:
CMD_VOLENVPER = 0
CMD_DUTYLL = 1
CMD_MSB = 2
CMD_LSB = 3
CMD_COUNT = 4

def onehot_cmd(data):
    cmd = data[CMD_OFFSET]
    nd = [ 0, 0, 0, 0 ]
    nd[int(cmd)] = 1
    return nd


CH_1 = 1
CH_2 = 2
CH_COUNT = 2

TIME_OFFSET = 0
CH_OFFSET = 1
CMD_OFFSET = 2
CHANNEL_OFFSET = 3
PARAM1_OFFSET = 4
PARAM2_OFFSET = 5
PARAM3_OFFSET = 6
SIZE_OF_INPUT_FIELDS = 7

WINDOW_SIZE = 256

M_CYCLES_PER_SECOND = 4194304.
NORMALIZE_TIME_BY = M_CYCLES_PER_SECOND * 3.

def norm(val, max_val):
    if val > max_val:
        return 1.
    else:
        return ((val / max_val) * 2.) - 1.

def unnorm(val, max_val):
    return ((val + 1.) / 2.) * max_val

def fresh_input(command, channel, time):
    newd = np.zeros(shape=SIZE_OF_INPUT_FIELDS, dtype=float)
    newd[TIME_OFFSET] = norm(time, NORMALIZE_TIME_BY)

    if int(channel) == 1:
        newd[CH_OFFSET] = norm(CH_1, CH_COUNT)
    elif int(channel) == 2:
        newd[CH_OFFSET] = norm(CH_2, CH_COUNT)
    else:
        raise "I didn't expect this"

    newd[CMD_OFFSET] = norm(channel, CMD_COUNT)
    return newd

def nop():
    return fresh_input(NOP_CMD_OFFSET, 1, 0)

def norm_command_of_parts(command, channel, parts, time):
    inp = fresh_input(command, channel, time)
    
    if command == CMD_DUTYLL:
        inp[PARAM1_OFFSET] = norm(float(parts[3]), 2.)
        inp[PARAM2_OFFSET] = norm(float(parts[4]), 64.)
    elif command == CMD_VOLENVPER:
        inp[PARAM1_OFFSET] = float(parts[3]) / 16.
        inp[PARAM2_OFFSET] = float(parts[4])
        inp[PARAM3_OFFSET] = float(parts[4]) / 7.
    elif command == CMD_LSB:
        inp[PARAM1_OFFSET] = norm(float(parts[3]), 255.)
        inp[PARAM2_OFFSET] = 0.
        inp[PARAM3_OFFSET] = 0
    elif command == CMD_MSB:
        inp[PARAM1_OFFSET] = norm(float(parts[3]), 7.)
        inp[PARAM2_OFFSET] = float(bool(parts[4]))
        inp[PARAM3_OFFSET] = float(bool(parts[5]))
    else:
        raise "this should not happen"
        
    return inp

def unnorm_feature(data):
    def l_unnorm(channel, maxv):
        data[channel] = unnorm(data[channel], maxv)
    l_unnorm(TIME_OFFSET, NORMALIZE_TIME_BY)
    # 4 cycles is the minimum distance between time points
    data[TIME_OFFSET] = max(data[TIME_OFFSET], 4)
    
    l_unnorm(CH_OFFSET, CH_COUNT)
    data[CH_OFFSET] = round(data[CH_OFFSET])
    
    l_unnorm(CMD_OFFSET, CMD_COUNT)
    data[CMD_OFFSET] = round(data[CMD_OFFSET])
    
    return data

def load_training_data(src):
    data = []
    file = open(src, 'r')
    for line in file:
        parts = line.split()
        if len(parts) > 0 and parts[0] == "CH":
            #print(parts)
            channel = int(parts[1])
            command = parts[2]
            time = int(parts[-1])
            if command == "DUTYLL":
                new_item = norm_command_of_parts(CMD_DUTYLL, channel, parts, time)
            elif command == "VOLENVPER":
                new_item = norm_command_of_parts(CMD_VOLENVPER, channel, parts, time)
            elif command == "FREQLSB":
                new_item = norm_command_of_parts(CMD_LSB, channel, parts, time)
            elif command == "FREQMSB":
                new_item = norm_command_of_parts(CMD_MSB, channel, parts, time)
             # Otherwise unknown   
            data.append(new_item)
           #print("NEXTCMD", data[-1])
    return data

@pescador.streamable
def samples_from_training_data(src, window_size=WINDOW_SIZE):
    sample_data = None

    try:
        sample_data = load_training_data(src)
    except Exception as e:
        LOGGER.error('Could not load {}: {}'.format(src, str(e)))
        raise StopIteration()

    true_window_size = window_size + 1

    # Pad small samples with nop
    while len(sample_data) < true_window_size:
        sample_data.append(nop())

    while True:

        if len(sample_data) == true_window_size:
            sample = sample_data
        else:
            # Sample a random window from the audio file
            start_idx = np.random.randint(0, len(sample_data) - true_window_size)
            end_idx = start_idx + true_window_size
            sample = sample_data[start_idx:end_idx]

        sample_input = sample[0:window_size]
        sample_output = sample[window_size:window_size+1]

        sample_input = np.array(sample_input).astype(np.float32)
        sample_output = np.array(sample_output).astype(np.float32)

        yield { 'X':sample_input, 'Y': sample_output }

def create_batch_generator(paths, batch_size):
    streamers = []
    for path in paths:
        print("Creating a batch generator")
        streamers.append(samples_from_training_data(path))
        print("Done creating batch generator")
    mux = pescador.ShuffledMux(streamers)
    batch_gen = pescador.buffer_stream(mux, batch_size)
    return batch_gen

def training_files(dirp):
    return [
      os.path.join(root, fname)
      for (root, dir_names, file_names) in os.walk(dirp, followlinks=True)
      for fname in file_names
    ]

def create_data_split(paths, batch_size):
    train_gen = create_batch_generator(paths, batch_size)
    return train_gen



In [7]:
print("Collecting training data")
train_gen = create_data_split(training_files("../..//training_data/"), 1)
print("Collected")

Creating a batch generator
Done creating batch generator
Creating a batch generator
Done creating batch generator
Creating a batch generator
Done creating batch generator
Creating a batch generator
Done creating batch generator
Creating a batch generator
Done creating batch generator
Creating a batch generator
Done creating batch generator
Creating a batch generator
Done creating batch generator
Creating a batch generator
Done creating batch generator
Creating a batch generator
Done creating batch generator
Creating a batch generator
Done creating batch generator
Creating a batch generator
Done creating batch generator
Creating a batch generator
Done creating batch generator
Creating a batch generator
Done creating batch generator
Creating a batch generator
Done creating batch generator
Creating a batch generator
Done creating batch generator
Creating a batch generator
Done creating batch generator
Creating a batch generator
Done creating batch generator
Creating a batch generator
Done

In [ ]:
DIM = SIZE_OF_INPUT_FIELDS * WINDOW_SIZE

class CommandNet(nn.Module):

    def __init__(self):
        super(CommandNet, self).__init__()

        self.main = nn.Sequential(
            nn.BatchNorm1d(WINDOW_SIZE),
            nn.Dropout(p=0.2),
            nn.Conv2d(1, 1, kernel_size=(SIZE_OF_INPUT_FIELDS, 4)),
            nn.Flatten(),
            nn.Conv1d(1, 1, kernel_size=SIZE_OF_INPUT_FIELDS),
            nn.Dropout(p=0.2),
            nn.Linear(994, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.Sigmoid(),
            nn.Linear(256, SIZE_OF_INPUT_FIELDS),
        )

    def forward(self, sequence):
        output = self.main(sequence)
        return output


EPOCHS = 2000
ROUND_SZ = 20000

def train():

    existing_path = None
    command_generator = CommandNet()
    
    if existing_path != None:
        command_generator.load_state_dict(torch.load(existing_path))
        
    command_generator = command_generator.to(device)

    command_optimizer= optim.Adam(command_generator.parameters(), lr=0.0001, weight_decay=1e-5)
    command_criterion = nn.MSELoss()
    
    decay1 = optim.lr_scheduler.ExponentialLR(command_optimizer, gamma=0.999)

    for iteration in range(EPOCHS):
        print(f"Round {iteration}")

        for i in range(ROUND_SZ):

          command_optimizer.zero_grad()

          data = next(train_gen)
          data_train_cmd = torch.Tensor(data['X']).to(device)
          data_test_cmd = torch.Tensor(data['Y'][0]).to(device)
            
          prediction_command = command_generator(data_train_cmd)
          #print(prediction_command.flatten(), data_test_command.flatten())
          command_loss = command_criterion(prediction_command, data_test_cmd)
          command_loss.backward()
          command_optimizer.step()


        print("Command batch loss:", command_loss.item())
        print("Last data:", unnorm_feature(data_test_cmd.detach().cpu().numpy()[0]))
        print("Last prediction:", unnorm_feature(prediction_command.detach().cpu().numpy()[0]))
        torch.save(command_generator.state_dict(), "./" + str(int(datetime.now().timestamp())) + ".checkpoint.model")
        print("Saved checkpoint")
        
        decay1.step()

    return data['X'][0], command_generator.eval()

seed, command_generator = train()

Round 0
Command batch loss: 0.0068533169105648994
Last data: [28.125       1.          1.          0.          0.42857143  1.
  1.        ]
Last prediction: [31367.25           1.             1.             0.00192593
     0.53640014     0.8551078      0.90162975]
Saved checkpoint
Round 1
Command batch loss: 0.18036523461341858
Last data: [592.125       1.          1.          0.         -0.9607843   0.
   0.       ]
Last prediction: [4.         1.         1.         0.00188623 0.03750512 0.2702093
 0.21645547]
Saved checkpoint
Round 2
Command batch loss: 0.2467152625322342
Last data: [71332.125     1.        1.        0.        1.       -1.        0.   ]
Last prediction: [103711.875           1.              1.              0.00143044
      0.6609226       0.14032806      0.39172837]
Saved checkpoint
Round 3
Command batch loss: 0.020517567172646523
Last data: [28.125  1.     1.     0.     1.     1.     1.   ]
Last prediction: [14871.75           1.             1.             0.0029183

Command batch loss: 0.0015040540602058172
Last data: [16.125  1.     1.     0.     0.375  0.     0.   ]
Last prediction: [6724.875         1.            1.            0.00103602    0.4737506
    0.00010899   -0.02355104]
Saved checkpoint
Round 32
Command batch loss: 0.0005366562982089818
Last data: [31.875       1.          1.          0.          0.42857143  1.
  1.        ]
Last prediction: [ 4.          1.          1.         -0.00036701  0.3788336   1.0166888
  1.0313729 ]
Saved checkpoint
Round 33
Command batch loss: 0.0008273495477624238
Last data: [19.875  2.     2.     0.     1.    -1.     0.   ]
Last prediction: [ 4.          2.          2.          0.00127314  0.93674344 -1.0037365
 -0.01203388]
Saved checkpoint
Round 34
Command batch loss: 0.0014718422899022698
Last data: [31.875  1.     1.     0.    -1.     0.     0.   ]
Last prediction: [211717.5             1.              1.             -0.00052255
     -0.9593871       0.01670722     -0.02870818]
Saved checkpoint
Round 

Command batch loss: 0.03200821578502655
Last data: [65563.875          2.             2.             0.
     0.94509804     0.             0.        ]
Last prediction: [209608.12            2.              2.              0.00035333
      0.4771077      -0.03556806     -0.00562216]
Saved checkpoint
Round 63
Command batch loss: 0.0012860088609158993
Last data: [52.125  2.     2.     0.     1.     1.     1.   ]
Last prediction: [ 4.          2.          2.         -0.00150876  0.91586727  0.9700697
  0.98236555]
Saved checkpoint
Round 64
Command batch loss: 6.361850682878867e-05
Last data: [19.875  2.     2.     0.     0.75   0.     0.   ]
Last prediction: [22573.125          2.             2.            -0.00082632
     0.7424825     -0.01239703    -0.00826009]
Saved checkpoint
Round 65
Command batch loss: 0.0003282022080384195
Last data: [24.          2.          2.          0.         -0.03529412  0.
  0.        ]
Last prediction: [46885.875          2.             2.            -0.00

Command batch loss: 0.002504279837012291
Last data: [16.125  1.     1.     0.     1.     1.     1.   ]
Last prediction: [1005.75          1.            1.           -0.00012393    0.962223
    1.0639693     1.038677  ]
Saved checkpoint
Round 93
Command batch loss: 0.0012468647910282016
Last data: [288.      2.      2.      0.      0.625   0.      0.   ]
Last prediction: [27698.25           2.             2.            -0.00006187
     0.53810686     0.01410449     0.00482623]
Saved checkpoint
Round 94
Command batch loss: 0.036874882876873016
Last data: [66348.        2.        2.        0.        0.875     0.        0.   ]
Last prediction: [4.         2.         2.         0.00224188 0.52678055 0.04812048
 0.06789383]
Saved checkpoint
Round 95
Command batch loss: 0.027544241398572922
Last data: [592.125        2.           2.           0.           0.01960784
   0.           0.        ]
Last prediction: [ 4.          2.          2.          0.00002268  0.45676756  0.00811609
 -0.006022

In [ ]:
seed = next(train_gen)['X']
for i in range(10000):
    pred = command_generator(torch.Tensor(seed).to(device))
    pred = pred.detach().cpu().numpy()
    print("Pred:", unnorm_feature(pred[0]))
    seed = np.array([np.append(seed[0][1:], pred, axis=0)])